#### ⏰ Até 15/02 ⏰ 1ª FASE - EXTRAÇÃO DOS DADOS DA ESCALA DE SERVIÇO 

In [ ]:
from docx import Document
import pandas as pd

# função que abre e extrai todas as tabelas de um docx
def extrairTabelas(caminhoDocx):
    todasTabelas = []

    doc = Document(caminhoDocx)
    
    for tabela_idx, tabela in enumerate(doc.tables):
        tabelaAtual = []
        
        for row_idx, row in enumerate(tabela.rows):
            cells = [cell.text.strip() for cell in row.cells]
            tabelaAtual.append(cells)

        todasTabelas.append(tabelaAtual)

    return todasTabelas

# coloca o nome do docx
tabelas = extrairTabelas('17.02.2024 ESCALA DE SERVIÇO - SÁBADO.docx')

# nomes das equipes de serviço para referenciar os dfs
nomesEquipes = ['dias','cpu','sede','liberdade','alemanha','gtm']

# cria o dicionário equipe-tabela a ser preenchido
dfEquipe = {}

# criação dos DataFrames a partir da extração em docx
for idx, tabela in enumerate(tabelas):
    if idx >= len(nomesEquipes):
        novaEquipe = tabela[0][0][:15].lower()
        nomesEquipes.append(novaEquipe)
    dfEquipe[nomesEquipes[idx]] = pd.DataFrame(tabela)
    display(dfEquipe[nomesEquipes[idx]])

#### ⏰ Até 20/02 ⏰ 2ª FASE - EXTRAÇÃO DOS DADOS DO CHECKLIST 

In [ ]:
import gspread

# armazenar JSON em local seguro após a fase de teste. Credenciais de Acesso por Conta de Serviço
gc = gspread.service_account('./service_account.json')

spreadsheet = gc.open_by_url('https://docs.google.com/spreadsheets/d/1k6Ugpr-XRiWQEFeNPEkGSODZW2vfx1m1r3GCNk8xnQo')

planilhaMotos = spreadsheet.get_worksheet(0)
dadosMotos = pd.DataFrame(planilhaMotos.get_all_values()[-100:])
dadosMotos = dadosMotos[[0,2,3,4,5,6]]
display(dadosMotos)

planilhaCarros = spreadsheet.get_worksheet(1)
dadosCarros = pd.DataFrame(planilhaCarros.get_all_values()[-100:])
dadosCarros = dadosCarros[[0,2,3,4,5,6]]
display(dadosCarros)

In [ ]:
inicioIntervalo = '17/02/2024 07:00:00'
fimIntervalo = '18/02/2024 07:00:00'

dadosMotosFiltrados = dadosMotos[(dadosMotos[0] >= inicioIntervalo) & (dadosMotos[0] <= fimIntervalo)]
display(dadosMotosFiltrados)

dadosCarrosFiltrados = dadosCarros[(dadosCarros[0] >= inicioIntervalo) & (dadosCarros[0] <= fimIntervalo)]
display(dadosMotosFiltrados)

In [ ]:
def exibirDados(valor, linhaNome, colunaId):
    if valor in dadosCarrosFiltrados[3].values:
        display(dadosCarrosFiltrados.loc[dadosCarrosFiltrados[3] == valor])
    elif valor in dadosMotosFiltrados[3].values:
        display(dadosMotosFiltrados.loc[dadosMotosFiltrados[3] == valor])
    else:
        mensagem = equipeAtual.loc[equipeAtual[colunaId] == valor][linhaNome].iloc[0]
        print(f'{mensagem} não fez checklist')
        
def processarEquipe(equipeAtual, colunaId, linhaNome, turno):
    for valor in equipeAtual[colunaId][3:]:
        exibirDados(valor, linhaNome, colunaId)



for i in range(len(nomesEquipes)):
    equipeAtual = dfEquipe[nomesEquipes[i]]
    if nomesEquipes[i] not in ['dias', 'expediente admi', 'sede']:
        processarEquipe(equipeAtual, 3, 2, '1ª Equipe')

        if equipeAtual[3][1] == equipeAtual[8][1]:
            processarEquipe(equipeAtual, 8, 7,'1ª Equipe')
                               
        else:
            processarEquipe(equipeAtual, 8, 7, '2ª Equipe')
           

#### ⏰ Até 25/02 ⏰ 3ª FASE - COMPARAÇÃO DADOS DA ESCALA E CHECKLIST EXECUTADOS 

#### ⏰ Até 1ª/03 ⏰ 4ª FASE - RELATÓRIO AO CHEFE DA 4ª SEÇÃO 